# Notebook 2 - Searching for solutions

CSI4106 Artificial Intelligence  
Fall 2020  
Prepared by Julian Templeton and Caroline Barrière

***INTRODUCTION***:   
We will look at the implementation of a search space.  The code below is largely inspired from the resources provided with the book Artificial Intelligence, Foundations of Computational Agents (https://artint.info/2e/online.html).  The resources corresponding to Chapter 3, on Searching for Solutions, are found here https://artint.info/AIPython/.  

There is even a companion Python book, with explanation of all the python code accompanying the book, as well as a first chapter called **Python for AI** where they give some basics of Python.  See https://artint.info/AIPython/aipython.pdf

The code below is more specifically based on the code in the file *searchProblem.py*.  You do not need to go back to that code. Everything you need is specified in the notebook.  I have modified the original code, and I'm presenting it here in small chunks so as to allow students less familiar with python to go through slowly. 

***HOMEWORK***:  
Go through the notebook by running each cell, one at a time. If you see an error, you can fix the issue and re-run the cells.  
Look for **(TO DO)** for the tasks that you need to perform.  Once you're done, submit your notebook. 

*The notebook will be marked on 20.  
Each **(TO DO)** has a number of points associated with it.*
***

**1. Definition of a directed graph.**  
A directed graph is made of nodes and arcs. The class below defines an arc that connects two nodes. The first two parameters give the two connected nodes. The other two parameters provide optional cost of an arc (with default to 1) and an optional action of the arc (default to None). The init is the constructor. The assert states a condition that must be true and provides what to do otherwise.

In [3]:
class Arc(object):
    """An arc has a from_node and a to_node node and a (non-negative) cost"""
    
    def __init__(self, from_node, to_node, cost=1, action=None):
        assert cost >= 0, ("Cost cannot be negative for"+
                           str(from_node)+"->"+str(to_node)+", cost: "+str(cost))
        self.from_node = from_node
        self.to_node = to_node
        self.action = action
        self.cost=cost

    def __repr__(self):
        """string representation of an arc"""
        if self.action:
            return str(self.from_node)+" --"+str(self.action)+"--> "+str(self.to_node)
        else:
            return str(self.from_node)+" --> "+str(self.to_node)

Here is an example of a small DAG (Directed Acyclic Graph). <img src="Image 1.png" />

In [4]:
# Defining a small dag
dag1 = [Arc('a','b'), Arc('b','c1'), Arc('c1','d'), Arc('b','c2'), Arc('c2','d')]

In [5]:
# print the dag
print(dag1)

[a --> b, b --> c1, c1 --> d, b --> c2, c2 --> d]


The Node class is not defined.  So far, it is assumed that a Node will simply be a string.

**2. Search Space **  
Instead of describing a dag as simply a set of arcs, we can more formally define a class that will represent a search problem which will contain not only the list of arcs, but the explicit list of nodes, the actual start node and the goals.  This class also contains useful methods to "query" the search problem for a node's neighbors, or for determining if a node is a goal or not. It also allows to include a heuristic function, which we will use later.

In [6]:
class Search_problem_from_explicit_graph():
    """A search problem consists of:
    * a list or set of nodes
    * a list or set of arcs
    * a start node
    * a list or set of goal nodes
    * a dictionary that maps each node into its heuristic value.
    """

    def __init__(self, nodes, arcs, start=None, goals=set(), hmap={}):
        self.neighs = {}
        self.nodes = nodes
        for node in nodes:
            self.neighs[node]=[]
        self.arcs = arcs
        for arc in arcs:
            self.neighs[arc.from_node].append(arc)
        self.start = start
        self.goals = goals
        self.hmap = hmap

    def start_node(self):
        """returns start node"""
        return self.start
    
    def is_goal(self,node):
        """is True if node is a goal"""
        return node in self.goals

    def neighbors(self,node):
        """returns the neighbors of node"""
        return self.neighs[node]

    def heuristic(self,node):
        """Gives the heuristic value of node n.
        Returns 0 if not overridden in the hmap."""
        if node in self.hmap:
            return self.hmap[node]
        else:
            return 0
        
    def __repr__(self):
        """returns a string representation of the search problem"""
        res=""
        for arc in self.arcs:
            res += str(arc)+".  "
        return res

    def neighbor_nodes(self,node):
        """returns an iterator over the neighbors of node"""
        return (path.to_node for path in self.neighs[node])


Below is an example of a search problem that we might want to represent. Assuming we wish to go from node 'a' to 'h' (highlighted in yellow). Under the figure is the definition.



<img src="Image 2.png"/>

In [7]:
# Defining a search space
problemSimple = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j'},
    [Arc('a','b',1), Arc('a','c',1), Arc('b','d',1), Arc('b','e',1),
     Arc('c','f',1), Arc('c', 'g',1), Arc('d','h',1), Arc('e','h',1), 
     Arc('f', 'e', 1), Arc('f','j',1)],
    start = 'a',
    goals = {'h'})

In [8]:
# Print some information
print(problemSimple.neighbors('a'))

[a --> b, a --> c]


**(TO DO) Q1 - 2 marks**  
Print the start node. Test if node 'g' is a goal node.  Go through the list of nodes to find which one is the goal.

In [41]:
# Print the start node. 
print('Part1: The start node is:',problemSimple.start,'\n')
# Test if node 'g' is a goal node. 
if (problemSimple.is_goal('g')) == True :
    print('g is a goal node')
else :
    print('Part2: g is not a goal node. The goal node is:', problemSimple.goals,'\n')
# Go through the list of nodes to find which one is the goal.
print('Part3:')
for node in problemSimple.nodes :
    if problemSimple.is_goal(node) == True :
        print(node,'is a goal node')
    else :
        print(node,'is not a goal node')

Part1: The start node is: a 

Part2: g is not a goal node. The goal node is: {'h'} 

Part3:
f is not a goal node
b is not a goal node
c is not a goal node
g is not a goal node
h is a goal node
e is not a goal node
a is not a goal node
j is not a goal node
d is not a goal node


**(TO DO) Q2 - 1 mark** <br>
Complete the definition of the problem space below, according to the following DAG.  The cost are not all equal to 1 now, but are rather given on the arcs.

<img src="Image 3.png" />

In [9]:
problemTest = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j', 'k', 'm', 'n'},
    [Arc('a','b',1), Arc('a','c',2), Arc('b','d',3), Arc('b','e',4),
     Arc('c','f',1), Arc('c','g',1), Arc('f','e',1), Arc('d','h',1), Arc('e','h',2), 
     Arc('f', 'j', 5), Arc('h','k',1), Arc('h','m',4), Arc('j','n',2), Arc('m','n',3)],
    start = 'a',
    goals = {'n'})

print(problemTest.neighbors('a'))

[a --> b, a --> c]


**3. Definition of a solution as a path.**  
A solution to a search problem is usually a Path, providing the set of visited nodes and arcs from source node to goal node.  Below a path is defined as either a single node (when arc=None), or a path followed by an arc.  Do not worry too much about all the implementation details in this class.  As your python skills get better, it will become more clear.

In [10]:
# class to represent a path
class Path(object):
    """A path is either a node or a path followed by an arc"""
    
    def __init__(self,initial,arc=None):
        """initial is either a node (in which case arc is None) or
        a path (in which case arc is an object of type Arc)"""
        self.initial = initial
        self.arc=arc
        if arc is None:
            self.cost=0
        else:
            self.cost = initial.cost+arc.cost

    def end(self):
        """returns the node at the end of the path"""
        if self.arc is None:
            return self.initial
        else:
            return self.arc.to_node

    def nodes(self):
        """enumerates the nodes for the path.
        This starts at the end and enumerates nodes in the path backwards."""
        current = self
        while current.arc is not None:
            yield current.arc.to_node
            current = current.initial
        yield current.initial

    def initial_nodes(self):
        """enumerates the nodes for the path before the end node.
        This starts at the end and enumerates nodes in the path backwards."""
        if self.arc is not None:
            for nd in self.initial.nodes(): yield nd     # could be "yield from"
        
    def __repr__(self):
        """returns a string representation of a path"""
        if self.arc is None:
            return str(self.initial)
        elif self.arc.action:
            return (str(self.initial)+"\n   --"+str(self.arc.action)
                    +"--> "+str(self.arc.to_node))
        else:
            return str(self.initial)+" --> "+str(self.arc.to_node)

**4.  Definition of a generic searcher**  
We first define a class representing a generic searcher, but with no strategy as to which node will be explored next. 

In [11]:
# (CB) Modified from book
class GenericSearcher():
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()

    def initialize_frontier(self):
        self.frontier = [Path(self.problem.start_node())]
        
    def empty_frontier(self):
        return self.frontier == []
       
    ### change the add_to_frontier method --- THIS METHOD NEEDS TO BE IMPLEMENTED FOR DIFFERENT SEARCH STRATEGIES
    def add_to_frontier(self,path):
        raise NotImplementedError
        
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            # HERE -- The next node explored is selected
            # It is implemented with a Pop, which actually removes from the end of the list
            path = self.frontier.pop()            
            print(path)
            if self.problem.is_goal(path.end()):    # solution found
                self.solution = path                # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())              
                for arc in neighs:
                    self.add_to_frontier(Path(path,arc))              

**5. Let's implement the three blind searchers.**  
As we saw in class, the behavior of a searcher depends on how the frontier is dealt with.  We refine the generic searcher into three specific searchers.  The three blind searchers we implement are depth-first, breadth-first and lowest-cost.  The breadth-first is implemented below.  You are asked to implement the other two.

In [12]:
# (CB) extend the generic searcher to be a Breadth-first searcher.
class BreadthFirstSearcher(GenericSearcher):
    
    def __init__(self, problem):   
        super().__init__(problem)

    def add_to_frontier(self,path):
        # breadth
        self.frontier.insert(0,path)        # HERE, I insert in position 0, knowing that I want a FIFO

In [47]:
# Test breadth-first searcher
searcherBreadth = BreadthFirstSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherBreadth.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> b --> d
a --> b --> e
a --> c --> f
a --> c --> g
a --> b --> d --> h
a --> b --> e --> h
a --> c --> f --> e
a --> c --> f --> j
a --> b --> d --> h --> k
a --> b --> d --> h --> m
a --> b --> e --> h --> k
a --> b --> e --> h --> m
a --> c --> f --> e --> h
a --> c --> f --> j --> n
Path found: a --> c --> f --> j --> n with a cost of 10


**(TO DO) Q3 - 2 marks** <br> 
Implement the add_to_frontier method for the depth-first search.  And test your implementation.

In [13]:
# Depth-first searcher
class DepthFirstSearcher(GenericSearcher):
    def __init__(self, problem):   
        super().__init__(problem)

    def add_to_frontier(self,path):
        # depth - put code here
        self.frontier.append(path)  # HERE, I append at the end of path, knowing that I want a LIFO

In [69]:
# Test
searcherDepth = DepthFirstSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherDepth.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> c
a --> c --> g
a --> c --> f
a --> c --> f --> j
a --> c --> f --> j --> n
Path found: a --> c --> f --> j --> n with a cost of 10


**(TO DO) Q4 - 5 marks** <br> 
Implement the add_to_frontier method for the lowest-cost search. And test your implementation.

In [23]:
# Lowest Cost searcher
class LowCostSearcher(GenericSearcher):
    def __init__(self, problem):   
        super().__init__(problem)

    def add_to_frontier(self,path):
        # Need to insert path p onto the frontier directly before the next
        # min path (if any).
        # low-cost search  - put code here...
        if self.frontier == [] :
            self.frontier.insert(0,path)
        else:
            mycost=path.cost
            ii=0
            found=False
            while ii<len(self.frontier) and found==False:
                if mycost>self.frontier[ii].cost:
                    ii=ii+1
                else:
                    found=True
                    
            if ii==len(self.frontier):
                self.frontier.append(path)
            else:
                self.frontier.insert(ii,path)

In [22]:
# Test
searcherLowCost = LowCostSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherLowCost.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> c
a --> c --> f
a --> c --> f --> j
a --> c --> f --> j --> n
Path found: a --> c --> f --> j --> n with a cost of 10


**6. Heuristic searches**  
We have seen three types of heuristic searches in class: greedy, best-first and A*.  We can implement them, as different specific searchers which will modify the add_to_frontier method of the generic searcher.    

Within this notebook, to get the heuristic of a node, you must call the self.problem.heuristic(...) function where you pass in the node that you want the heuristic for. An example of its usage can be found in 6 (b)'s implementation of the A* searcher.

Let's assume the heuristic function provides an optimistic distance of each node to the goal node 'n'. These distances are encoded in a dictionary call hmap, in the problem definition below, which is an extension to problemTest defined earlier.

In [129]:
problemTestWithHeuristics = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j', 'k', 'm', 'n'},
    [Arc('a','b', 1), Arc('a','c', 2), Arc('b','d',3), Arc('b','e',4),
     Arc('c','f',1), Arc('c','g',1), Arc('d', 'h', 1), Arc('e','h',2), 
     Arc('f','e',1), Arc('f','j',5), Arc('h', 'k',1), Arc('h','m',4), 
     Arc('m','n',3), Arc('j','n',2)],
    start = 'a',
    goals = {'n'},
     hmap = {'a':15, 'b':3, 'c':5, 'd':6, 'e':3, 'f':1, 'g':2, 'h':4, 'j':9, 'k':3, 'm':2, 'n':0})

***6 (a). Local heuristic search***    
As discussed in the lectures, there are two types of heuristic search algorithms. The first type is local heuristic searches. The local heuristic search algorithm discussed in class is the Greedy search, which assumes a limited view of a problem.     

**(TO DO) Q5 - 5 marks** <br> 
For your first heuristic algorithm, implement and test the Greedy search local heuristic search algorithm in the cells below. Recall that the greedy algorithm looks at the local costs. Furthermore, the greedy search algorithm will need to backtrack if it is at a dead-end. This means that if a path results in a dead-end, the search algorithm must continue the search from the previous node in that path.

In [197]:
# Greedy searcher
class GreedySearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)
    
    def add_to_frontier(self,path):
        # greedy search - put code here...
    
        f_len = len(self.frontier)
        Entered = False
        # I am using while loop to sort the frontier because I could not find suitable parameters for a built-in sort command.
        # However, my while loop sorts the frontier as desired.
        while Entered == False and f_len > 0 :
            A_Boolean = (self.frontier[f_len-1].initial.end() == path.initial.end())
            B_Boolean = (self.frontier[f_len-1].arc.cost < path.arc.cost)
            if A_Boolean and B_Boolean : 
                f_len = f_len-1
            else:
                Entered = True
                self.frontier.insert(f_len,path)
                
        if f_len == 0 and Entered == False :
            self.frontier.insert(0,path)        

In [198]:
# Test
searcherGreedy = GreedySearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherGreedy.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> b --> d
a --> b --> d --> h
a --> b --> d --> h --> k
a --> b --> d --> h --> m
a --> b --> d --> h --> m --> n
Path found: a --> b --> d --> h --> m --> n with a cost of 12


***6 (b). Global heuristic search***    
The second type of heuristic search are global heuristic searches. The A* search algorithm and Best-first search algorithm are both global heuristic search algorithms.    

Before you implement the Best-first searcher, below is the A* search algorithm to use as a reference.

In [199]:
class AStarSearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)

    def getCostKey(self, path):
        return (path.cost + self.problem.heuristic(path.end()))
    
    def add_to_frontier(self,path):
        """add path to the frontier with the appropriate cost"""
        # add path to the frontier 
        self.frontier.append(path)
        # sort frontier so that all the elements are ordered from most costly to least costly
        self.frontier.sort(key=self.getCostKey, reverse=True)

In [200]:
# Test
searcherAStar = AStarSearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherAStar.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> c --> f
a --> c --> g
a --> c --> f --> e
a --> b --> e
a --> c --> f --> e --> h
a --> c --> f --> e --> h --> k
a --> b --> d
a --> b --> d --> h
a --> b --> d --> h --> k
a --> b --> d --> h --> m
a --> b --> e --> h
a --> b --> e --> h --> k
a --> b --> d --> h --> m --> n
Path found: a --> b --> d --> h --> m --> n with a cost of 12


**(TO DO) Q6 - 5 marks** <br> 
With the A* searcher implemented above, implement and test the best-first searcher below.

In [201]:
# Best first searcher
class BestFirstSearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)
        
    def getCostKey(self, path):
        return self.problem.heuristic(path.end())
    
    def add_to_frontier(self,path):
        # best-first search - put code here...
        # add path to the frontier 
        self.frontier.append(path)
        # sort frontier so that all the elements are ordered from most costly to least costly
        self.frontier.sort(key=self.getCostKey, reverse=True)

In [202]:
# Test
searcherBestFirst = BestFirstSearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherBestFirst.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> b --> e
a --> b --> e --> h
a --> b --> e --> h --> m
a --> b --> e --> h --> m --> n
Path found: a --> b --> e --> h --> m --> n with a cost of 14


***SIGNATURE:***
My name is Fatemeh Soltani.
My student number is 300139153.
I certify being the author of this assignment.